# Figure panel 5

In [ ]:
options(warn=-1)

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 

            # TradeSeq
            library(tradeSeq),

            # CCI 
            library(CellChat), 

            # Data 
            library(tidyverse), 
            library(data.table), 

            # STRING
            library(STRINGdb), 
            library(igraph), 
            library(tidygraph), 
            library(ggraph), 

            # Plotting 
            library(ggplot2), 
            library(patchwork), 
            library(cowplot), 
            library(ComplexHeatmap), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [ ]:
# Configure reticulate 
# use_condaenv(condaenv="p.3.10.16", conda="C:/miniconda3/bin/conda", required=NULL)
# py_config()

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("C:/Users/Florian Deckert/Projects/FD20200109SPLENO/") 

In [ ]:
# Source
source("bin/so_pl.R")
source("bin/dea_pp.R")
source("bin/dea_pl.R")
source("bin/gsea_pp.R")
source("bin/gsea_pl.R")
source("bin/cci_pp.R")
source("bin/cci_pl.R")

In [ ]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=4)) # From project global source()

# Import scRNAseq data 

In [ ]:
so <- readRDS("data/scRNAseq/object/pp_1.rds")
so <- NormalizeData(so)

# Differential expression analysis (DEA)

In [ ]:
# Response gene DEA 
contrasts_vec <- c("IFNAR_fl_CpG_D1", "IFNAR_fl_Baseline_D0")
dea_res_1 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

contrasts_vec <- c("IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_LysM_cre_Baseline_D0")
dea_res_2 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

contrasts_vec <- c("IFNAR_fl_CpG_D3", "IFNAR_fl_Baseline_D0")
dea_res_3 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

contrasts_vec <- c("IFNAR_fl_LysM_cre_CpG_D3", "IFNAR_fl_LysM_cre_Baseline_D0")
dea_res_4 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

# Genotype DEA
contrasts_vec <- c("IFNAR_fl_LysM_cre_Baseline_D0", "IFNAR_fl_Baseline_D0")
dea_res_5 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

contrasts_vec <- c("IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_CpG_D1")
dea_res_6 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

contrasts_vec <- c("IFNAR_fl_LysM_cre_CpG_D3", "IFNAR_fl_CpG_D3")
dea_res_7 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

In [ ]:
adj_pval_thr <- 0.05
log2FC_thr <- 0.50

# DEG count overview

## Response count D1 and D3 vs Ctl (Heatmap)

In [ ]:
mat_1 <- lapply(dea_res_1, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 
mat_2 <- lapply(dea_res_2, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 
mat_3 <- lapply(dea_res_3, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 
mat_4 <- lapply(dea_res_4, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 

In [ ]:
mat <- list(mat_1, mat_2, mat_3, mat_4)

In [ ]:
mat <- lapply(mat, function(x) {
    
    x <- as.data.frame(x)
    set_diff <- setdiff(make.names(levels(so$celltype_low)), rownames(x))
    x[set_diff, ] <- 0
    x[make.names(levels(so$celltype_low)), , drop=FALSE]

}
             )

mat <- do.call(cbind, mat)

In [ ]:
colnames(mat) <- c("+/+", "cre/+", "+/+", "cre/+")
col_split <- c("D1", "D1", "D3", "D3")

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/hm_deg_count_d1_d3_vs_ctl_cg_and_ko.pdf", width=1.5, height=3)

draw(dea_count_hm(as.matrix(mat), col_split=col_split, fontsize_select=2))

dev.off()

## Genotype (Heatmap) 

In [ ]:
celltype_select <- levels(so$celltype_low) %>% make.names()

In [ ]:
mat_1 <- rbind(
        
    dplyr::bind_rows(dea_res_5, .id="celltype") %>% dplyr::mutate(dpi="Ctl"), 
    dplyr::bind_rows(dea_res_6, .id="celltype") %>% dplyr::mutate(dpi="D1"), 
    dplyr::bind_rows(dea_res_7, .id="celltype") %>% dplyr::mutate(dpi="D3")

) %>% 
    dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% 
    dplyr::group_by(celltype, dpi) %>% dplyr::summarise(n=n()) %>% tidyr::pivot_wider(., names_from=dpi, values_from=n, values_fill=0) %>% 
    tibble::column_to_rownames("celltype")

In [ ]:
set_diff <- setdiff(celltype_select, rownames(mat_1))
mat_1[set_diff, ] <- 0
mat_1 <- mat_1[celltype_select, c("Ctl", "D1", "D3"), drop=FALSE]

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/hm_deg_count_cg_vs_ko.pdf", width=1.5, height=3)

draw(dea_count_hm(as.matrix(mat_1), col_split=NULL, fontsize_select=2))

dev.off()

## Genotype (jitter plot)

In [ ]:
celltype_select <- c("MEP", "Proerythroblast", "Erythroblast", "cMo", "intMo", "RPM")

In [ ]:
mat_1 <- rbind(
    
    dplyr::bind_rows(dea_res_6, .id="celltype") %>% dplyr::mutate(dpi="D1"), 
    dplyr::bind_rows(dea_res_7, .id="celltype") %>% dplyr::mutate(dpi="D3")

    ) %>% dplyr::filter(celltype %in% celltype_select) %>% 

    dplyr::mutate(sig=ifelse(p_val_adj<=adj_pval_thr & avg_log2FC<=-log2FC_thr, "+/+", ifelse(p_val_adj<=adj_pval_thr & avg_log2FC>=+log2FC_thr, "cre/+", "n.s.")))

In [ ]:
counts_2 <- mat_1 %>% dplyr::filter(sig!="n.s.") %>% dplyr::group_by(dpi, celltype, sig) %>% dplyr::summarise(n=n()) %>% dplyr::arrange(celltype, dpi, sig)

In [ ]:
mat_1 <- mat_1 %>% dplyr::mutate(sign_log10_padj=-sign(avg_log2FC)*log10(p_val_adj)) %>% dplyr::mutate(sign_log10_padj=ifelse(abs(sign_log10_padj)>60, sign(avg_log2FC)*60, sign_log10_padj)) %>% 
    dplyr::mutate(avg_log2FC=ifelse(abs(avg_log2FC)>5, sign(avg_log2FC)*5, avg_log2FC))

In [ ]:
jp_1 <- ggplot(mat_1 %>% dplyr::filter(celltype==celltype_select[1]), aes(y=sign_log10_padj, x=dpi, color=sig)) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype==celltype_select[1] & sig=="n.s."), size=0.1) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype==celltype_select[1] & sig!="n.s."), size=0.1) + 
    geom_hline(yintercept=0) +
    ylim(-60, +60) + 
    scale_color_manual(values=rev(c("+/+"="#cd34b5", "cre/+"="#FFAC1E", "n.s."="#7f7f7f"))) + 
    ylab("") + xlab("") + 
    facet_grid(~dpi, scales="free") + theme(aspect.ratio=2) + theme_global_set(4)

jp_2 <- ggplot(mat_1 %>% dplyr::filter(celltype==celltype_select[2]), aes(y=sign_log10_padj, x=dpi, color=sig)) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype==celltype_select[2] & sig=="n.s."), size=0.1) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype==celltype_select[2] & sig!="n.s."), size=0.1) + 
    geom_hline(yintercept=0) +
    ylim(-60, +60) + 
    scale_color_manual(values=c("+/+"="#cd34b5", "cre/+"="#FFAC1E", "n.s."="#7f7f7f")) + 
    ylab("") + xlab("") + 
    facet_grid(~dpi, scales="free") + theme(aspect.ratio=2) + theme_global_set(4)

jp_3 <- ggplot(mat_1 %>% dplyr::filter(celltype==celltype_select[3]), aes(y=sign_log10_padj, x=dpi, color=sig)) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype==celltype_select[3] & sig=="n.s."), size=0.1) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype==celltype_select[3] & sig!="n.s."), size=0.1) + 
    geom_hline(yintercept=0) +
    ylim(-60, +60) + 
    scale_color_manual(values=c("+/+"="#cd34b5", "cre/+"="#FFAC1E", "n.s."="#7f7f7f")) + 
    ylab("") + xlab("") + 
    facet_grid(~dpi, scales="free") + theme(aspect.ratio=2) + theme_global_set(4)

jp_4 <- ggplot(mat_1 %>% dplyr::filter(celltype==celltype_select[4]), aes(y=sign_log10_padj, x=dpi, color=sig)) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype==celltype_select[4] & sig=="n.s."), size=0.1) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype==celltype_select[4] & sig!="n.s."), size=0.1) + 
    geom_hline(yintercept=0) +
    ylim(-60, +60) + 
    scale_color_manual(values=c("+/+"="#cd34b5", "cre/+"="#FFAC1E", "n.s."="#7f7f7f")) + 
    ylab("") + xlab("") + 
    facet_grid(~dpi, scales="free") + theme(aspect.ratio=2) + theme_global_set(4)

jp_5 <- ggplot(mat_1 %>% dplyr::filter(celltype==celltype_select[5]), aes(y=sign_log10_padj, x=dpi, color=sig)) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype==celltype_select[5] & sig=="n.s."), size=0.1) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype==celltype_select[5] & sig!="n.s."), size=0.1) + 
    geom_hline(yintercept=0) +
    ylim(-60, +60) + 
    scale_color_manual(values=c("+/+"="#cd34b5", "cre/+"="#FFAC1E", "n.s."="#7f7f7f")) + 
    ylab("") + xlab("") + 
    facet_grid(~dpi, scales="free") + theme(aspect.ratio=2) + theme_global_set(4)

jp_6 <- ggplot(mat_1 %>% dplyr::filter(celltype==celltype_select[6]), aes(y=sign_log10_padj, x=dpi, color=sig)) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype==celltype_select[6] & sig=="n.s."), size=0.1) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype==celltype_select[6] & sig!="n.s."), size=0.1) +  
    geom_hline(yintercept=0) +
    ylim(-60, +60) + 
    scale_color_manual(values=c("+/+"="#cd34b5", "cre/+"="#FFAC1E", "n.s."="#7f7f7f")) + 
    ylab("") + xlab("") + 
    facet_grid(~dpi, scales="free") + theme(aspect.ratio=2) + theme_global_set(4)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/jp_dea_count_genotype.pdf", width=3, height=6)

    gridExtra::grid.arrange(
            
        jp_1 %>% egg::set_panel_size(., width=unit(0.5, "cm"), height=unit(1.5, "cm")), 
        jp_2 %>% egg::set_panel_size(., width=unit(0.5, "cm"), height=unit(1.5, "cm")), 
        jp_3 %>% egg::set_panel_size(., width=unit(0.5, "cm"), height=unit(1.5, "cm")), 
        jp_4 %>% egg::set_panel_size(., width=unit(0.5, "cm"), height=unit(1.5, "cm")), 
        jp_5 %>% egg::set_panel_size(., width=unit(0.5, "cm"), height=unit(1.5, "cm")), 
        jp_6 %>% egg::set_panel_size(., width=unit(0.5, "cm"), height=unit(1.5, "cm")), 
        nrow=6
        
    )

dev.off() 

# Volcano plots  

## CtlKO vs Ctl

In [ ]:
labels <- list(
    
    "intMo"=c("Icam1", "Itga4", "Itgb1", "Vcam1", "Gdf15", "Ccl2", "Tnf", "Il6", "Tgfb1", "Stat1", "Stat2", "Spic", "Irf8", "Siglec1", "Slc40a1", "Hmox1", "Blvrb", "Treml4", "Bach1", "Il1a", "Il1b", "Il1rn", "Il18", "Il33")

)

In [ ]:
vp_5 <- lapply(names(dea_res_5), function(i) {

    v_pl_i <- v_pl(dea_res_5[[i]], title=i, log2FC_thr=log2FC_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_LysM_cre_Baseline_D0"]), color_neg=unlist(color$sample_group["IFNAR_fl_Baseline_D0"]), label=if(i %in% names(labels)) {labels[[i]]} else {label <- NULL}, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5, label_merge=TRUE) + theme(legend.position="none") + theme_global_set(size_select=4)

}
                 )

In [ ]:
vp_5 <- lapply(vp_5, function(p) {egg::set_panel_size(p, width=unit(3, "cm"), height=unit(2.5, "cm"))})
vp_5 <- do.call(gridExtra::arrangeGrob, c(vp_5, ncol=5, nrow=ceiling(length(vp_5)/5)))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/vp_ctlko_vs_ctl.pdf", width=2*5, height=2*ceiling(length(vp_5)/5))

grid::grid.draw(vp_5)

dev.off()

## D1KO vs D1

In [ ]:
labels <- list(
    
    "intMo"=c("Icam1", "Itga4", "Itgb1", "Vcam1", "Gdf15", "Ccl2", "Tnf", "Il6", "Tgfb1", "Stat1", "Stat2", "Spic", "Irf8", "Siglec1", "Slc40a1", "Hmox1", "Blvrb", "Treml4", "Bach1", "Il1a", "Il1b", "Il1rn", "Il18", "Il33")

)

In [ ]:
vp_6 <- lapply(names(dea_res_6), function(i) {

    v_pl_i <- v_pl(dea_res_6[[i]], title=i, log2FC_thr=log2FC_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D1"]), color_neg=unlist(color$sample_group["IFNAR_fl_CpG_D1"]), label=if(i %in% names(labels)) {labels[[i]]} else {label <- NULL}, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5, label_merge=TRUE) + theme(legend.position="none") + theme_global_set(size_select=4)

}
                 )

In [ ]:
vp_6 <- lapply(vp_6, function(p) {egg::set_panel_size(p, width=unit(3, "cm"), height=unit(2.5, "cm"))})
vp_6 <- do.call(gridExtra::arrangeGrob, c(vp_6, ncol=5, nrow=ceiling(length(vp_6)/5)))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/vp_d1ko_vs_d1.pdf", width=2*5, height=2*ceiling(length(vp_6)/5))

grid::grid.draw(vp_6)

dev.off()

## D3KO vs D3

In [ ]:
labels <- list(
    
    "intMo"=c("Icam1", "Itga4", "Itgb1", "Vcam1", "Gdf15", "Ccl2", "Tnf", "Il6", "Tgfb1", "Stat1", "Stat2", "Spic", "Irf8", "Siglec1", "Slc40a1", "Hmox1", "Blvrb", "Treml4", "Bach1", "Il1a", "Il1b", "Il1rn", "Il18", "Il33")

)

In [ ]:
vp_7 <- lapply(names(dea_res_7), function(i) {

    v_pl_i <- v_pl(dea_res_7[[i]], title=i, log2FC_thr=log2FC_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D3"]), color_neg=unlist(color$sample_group["IFNAR_fl_CpG_D3"]), label=if(i %in% names(labels)) {labels[[i]]} else {label <- NULL}, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5, label_merge=TRUE) + theme(legend.position="none") + theme_global_set(size_select=4)

}
                 )

In [ ]:
vp_7 <- lapply(vp_7, function(p) {egg::set_panel_size(p, width=unit(3, "cm"), height=unit(2.5, "cm"))})
vp_7 <- do.call(gridExtra::arrangeGrob, c(vp_7, ncol=5, nrow=ceiling(length(vp_7)/5)))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/vp_d3ko_vs_d3.pdf", width=2*5, height=2*ceiling(length(vp_7)/5))

grid::grid.draw(vp_7)

dev.off()

# DEA result heatmap with bulk expression matrix

In [ ]:
hm_dea_fetch <- function(celltype, row_genes, col_samples, col_split, col_label=NULL, scale_celltype=NULL, scale_mode="all") {

    # Subset by sample group 
    so_i <- so[, so$sample_group %in% col_samples & so$celltype_low %in% celltype, drop=FALSE] # Filtering celltype_low for performance but will not work with scale_mode all
    
    # Get pseudobulk counts 
    if(scale_mode=="all" & !is.null(scale_celltype)) {

        so_i <- so[, so$celltype_low %in% scale_celltype, drop=FALSE]
        
    } 
    
    # Get pseudobulk counts 
    mat_1 <- GetAssayData(so_i, assay="RNA", layer="counts")
    mat_1 <- split(t(mat_1) %>% as.data.frame(), f=paste(so_i$celltype_low, so_i$sample_group))
    mat_1 <- lapply(mat_1, colSums) %>% do.call(rbind, .) %>% t() 
    mat_1 <- t(t(mat_1) / colSums(mat_1))
    
    if(scale_mode=="all") {

        mat_1 <- apply(mat_1, 1, scales::rescale) %>% t() # It apply transforms the data frame
        mat_1 <- mat_1[unlist(row_genes), paste(celltype, col_samples)]

        
    } else if (scale_mode=="sample") {

        mat_1 <- mat_1[unlist(row_genes), paste(celltype, col_samples)]
        mat_1 <- apply(mat_1, 1, scales::rescale) %>% t() # It apply transforms the data frame
        
    }

    if(!is.null(col_label)) {

        colnames(mat_1) <- col_label
        
    }
    

    row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))
    row_split <- row_split[row_split %in% rownames(mat_1)]

    # DEA log2FC 
    mat_2 <- purrr::reduce(
        
        list(
            
            dea_res_5[[celltype]] %>% dplyr::mutate(Ctl=avg_log2FC) %>% dplyr::select(gene, Ctl), 
            dea_res_6[[celltype]] %>% dplyr::mutate(D1=avg_log2FC) %>% dplyr::select(gene, D1), 
            dea_res_7[[celltype]] %>% dplyr::mutate(D3=avg_log2FC) %>% dplyr::select(gene, D3)
        
        ), full_join, by="gene"
    
    ) %>% tibble::column_to_rownames("gene")

    mat_2[setdiff(row_split, rownames(mat_2)), ] <- 0
    mat_2 <- mat_2[row_split, ]
    mat_2[is.na(mat_2)] <- 0
    
    # DEA adj p-values
    mat_3 <- purrr::reduce(
        
        list(
            
            dea_res_5[[celltype]] %>% dplyr::mutate(Ctl=p_val_adj) %>% dplyr::select(gene, Ctl), 
            dea_res_6[[celltype]] %>% dplyr::mutate(D1=p_val_adj) %>% dplyr::select(gene, D1), 
            dea_res_7[[celltype]] %>% dplyr::mutate(D3=p_val_adj) %>% dplyr::select(gene, D3)
        
        ), full_join, by="gene"
    
    ) %>% tibble::column_to_rownames("gene")

    mat_3[setdiff(row_split, rownames(mat_3)), ] <- 1
    mat_3 <- mat_3[row_split, ]
    mat_3[is.na(mat_3)] <- 1
    mat_3[(mat_3>adj_pval_thr | abs(mat_2)<log2FC_thr)] <- 1

    # Row split
    row_split=names(row_split)
    row_split=factor(row_split, levels=names(row_genes))
    
    hm_1 <- dea_exp_hm(mat_1, col_split=col_split, row_split=row_split, scale_width=1, legend_title="Exp.", cluster_rows=TRUE, fontsize_select=2)
    hm_2 <- dea_res_hm(mat_2, mat_3, p_val_adj_thr=adj_pval_thr, row_split=row_split, scale_width=1, color_min="#cd34b5", color_max="#FFAC1E", cluster_rows=TRUE, fontsize_select=2)

    hm <- hm_2 + hm_1

    return(hm)
}

In [ ]:
adj_pval_thr <- 0.05
log2FC_thr <- 0.50

In [ ]:
col_samples <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_LysM_cre_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_CpG_D3", "IFNAR_fl_LysM_cre_CpG_D3")
col_split <- c("Ctl", "Ctl", "D1", "D1", "D3", "D3")
col_label <- c("+/+", "cre/+", "+/+", "cre/+", "+/+", "cre/+")

## cMo

In [ ]:
# genes_1 <- dea_res_6[["cMo"]] %>% dplyr::filter(abs(avg_log2FC)>=log2FC_thr & p_val_adj<=adj_pval_thr) %>% pull(gene)
# genes_2 <- dea_res_7[["cMo"]] %>% dplyr::filter(abs(avg_log2FC)>=log2FC_thr & p_val_adj<=adj_pval_thr) %>% pull(gene)

In [ ]:
# row_genes <- list("cMo"=c(genes_1, genes_2) %>% unique())

In [ ]:
# hm_1 <- hm_dea_fetch(celltype="cMo", row_genes=row_genes, col_samples=col_samples, col_split=col_split, col_label=col_label, scale_mode="sample")

In [ ]:
# pdf("result/figures/1_scRNAseq/panel_5/hm_dea_ctl_d1_d3_genotype_cmo_pb.pdf", width=5, height=50)

# hm_1

# dev.off()

## intMo

In [ ]:
# genes_1 <- dea_res_6[["intMo"]] %>% dplyr::filter(abs(avg_log2FC)>=log2FC_thr & p_val_adj<=adj_pval_thr) %>% pull(gene)
# genes_2 <- dea_res_7[["intMo"]] %>% dplyr::filter(abs(avg_log2FC)>=log2FC_thr & p_val_adj<=adj_pval_thr) %>% pull(gene)

In [ ]:
# row_genes <- list("intMo"=c(genes_1, genes_2) %>% unique())

In [ ]:
# hm_1 <- hm_dea_fetch(celltype="intMo", row_genes=row_genes, col_samples=col_samples, col_split=col_split, col_label=col_label, scale_mode="sample")

In [ ]:
# pdf("result/figures/1_scRNAseq/panel_5/hm_dea_ctl_d1_d3_genotype_intmo_pb.pdf", width=5, height=50)

# hm_1

# dev.off()

## cMo (selected)

In [ ]:
row_genes <- list(

    "G1"=c("Stat1", "Stat2", "Irf1", "Irf7", "Ifit1", "Ifit2", "Ifit3", "Isg15", "Ddx60", "Usp18", "Socs1", "Ccl2", "Cxcl10"), # Interferon stimulation 
    "G2"=c("Spic", "Hmox1", "Maf", "Mafb", "Axl", "C5ar1", "Fcgr1"), # Macrophage development
    "G3"=c("Ftl1", "Fth1", "Nr1h3", "Blvrb"), # Iron metabolism
    "G4"=c("Dusp1", "Nlrp3", "Atf3", "Jun", "Fos", "Ccl4", "Ccl3", "Tnf", "Il1b", "Tnfaip3", "Nfkbia") # MAPK activation (Dusp1), Nfkb inhibition (Tnfaip3)
    
)

In [ ]:
hm_1 <- hm_dea_fetch(celltype="cMo", row_genes=row_genes, col_samples=col_samples, col_split=col_split, col_label=col_label, scale_mode="sample")

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/hm_dea_ctl_d1_d3_genotype_cmo_pb_selected.pdf", width=5, height=10)

hm_1

dev.off()

## intMo (selected)

In [ ]:
row_genes <- list(

    "G1"=c("Stat1", "Stat2", "Irf7", "Ifit1", "Ifit2", "Ifit3", "Isg15", "Ddx60", "Usp18", "Socs1", "Ccl2", "Cxcl10", "Gbp2", "Gbp2b", "Gbp3", "Gbp5", "Gbp7", "Cxcl9", "Ciita"), # Interferon stimulation 
    "G2"=c("Spic", "Hmox1", "Maf", "Mafb", "Vcam1", "Msr1", "Axl", "C5ar1", "Fcgr1"), # Macrophage development
    "G3"=c("Slc40a1", "Slc48a1", "Ftl1", "Fth1", "Prdx1", "Nr1h3", "Blvrb", "Hfe"), # Iron metabolism
    "G4"=c("Dusp1", "Atf3", "Jun", "Fos", "Ccl4", "Ccl3", "Il18", "Tnfaip3", "Nfkbia"), # MAPK activation (Dusp1), Nfkb inhibition (Tnfaip3)
    "G5"=c("Eif2ak2", "Atf4", "Ppp1r15a", "Slc7a11", "Gdf15") # Integrated stress response. Eif2ak2 is PKR, a kinase activator of 
    
)

In [ ]:
hm_1 <- hm_dea_fetch(celltype="intMo", row_genes=row_genes, col_samples=col_samples, col_split=col_split, col_label=col_label, scale_mode="sample")

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/hm_dea_ctl_d1_d3_genotype_intmo_pb_selected.pdf", width=5, height=10)

hm_1

dev.off()

# Protein-Protein interaction 

In [ ]:
# Create STRING DB for mouse 
string_db <- STRINGdb::STRINGdb$new(version="12.0", species=10090, score_threshold=400, network_type="full", link_data="combined_only")

## intMo D1 KO UP 

In [ ]:
# Mapp DEA results to STRING DB
string_db_dea <- dea_res_6[["intMo"]] %>% dplyr::select(gene, avg_log2FC, p_val_adj) %>% dplyr::rename(logFC=avg_log2FC, padj=p_val_adj) %>% na.omit() %>% dplyr::filter(padj<=adj_pval_thr & logFC>=+log2FC_thr) %>% string_db$map(., "gene", removeUnmappedRows=TRUE)

In [ ]:
# Labe PPI vertices 
ppi_v <- string_db_dea %>% dplyr::distinct() %>% dplyr::rename(name=STRING_id) %>% 
    dplyr::group_by(name) %>% dplyr::mutate(n=n()) %>% dplyr::filter(n==1) %>% dplyr::select(name, gene, logFC, padj) %>% 
    dplyr::mutate(node_group=ifelse(padj<=0.05 & logFC<=-0.50, "neg", ifelse(padj<=0.05 & logFC>=0.50, "pos", "n.s.")))

In [ ]:
# Create PPI edges from DEA PPI
ppi_e <- string_db$get_interactions(string_db_dea$STRING_id)

In [ ]:
# Compute igraph object
g <- igraph::graph_from_data_frame(
    
    d=ppi_e,
    vertices=ppi_v, 
    directed=FALSE

)

In [ ]:
# Compute GO enrichment 
enrichment <- string_db$get_enrichment(V(g)$name, category="Process", methodMT="fdr", iea=TRUE)
openxlsx::write.xlsx(enrichment %>% dplyr::arrange(fdr), "result/string/scRNAseq/enrichment_intmo_d1_ko_up.xlsx")

In [ ]:
# Connected components 
min_component_size <- 4  # keep components with >= 4 vertices

g_components <- igraph::components(g)
g <- igraph::delete_vertices(g, which(g_components$csize[g_components$membership] < min_component_size))

In [ ]:
# Compute Louvain cluster on combined score 
E(g)$weight <- E(g)$combined_score

louvain <- igraph::cluster_louvain(g, weights=E(g)$weight)
V(g)$louvain <- membership(louvain)

In [ ]:
# Compute GO enrichment for Louvain 
enrichment_louvain <- tibble::tibble(name=V(g)$name, louvain=V(g)$louvain) %>% group_by(louvain) %>%
    
    group_modify(~{
        
        enrichment <- string_db$get_enrichment(.x$name, category="Process", methodMT="fdr", iea=TRUE) %>% arrange(fdr)
        if (is.null(enrichment) | nrow(enrichment)==0) {
            
            return(tibble::tibble())
        
        } else {

            return(enrichment %>% arrange(fdr))
            
        }
        
    }
                ) %>% ungroup()

In [ ]:
openxlsx::write.xlsx(enrichment_louvain %>% dplyr::filter(fdr<=0.01), "result/string/scRNAseq/enrichment_louvain_intmo_d1_ko_up.xlsx")

In [ ]:
# Fetch GO enrichment 
enrichment_label <- enrichment_louvain %>% group_by(louvain) %>% arrange(fdr) %>% slice(1) %>% ungroup() %>% transmute(louvain, label=description)
V(g)$enrichment_label <- enrichment_label$label[match(V(g)$louvain, enrichment_label$louvain)]

In [ ]:
# Remove vertices without enrichment label 
g <- igraph::delete_vertices(g, which(is.na(V(g)$enrichment_label)))

In [ ]:
# Plot graph 
g_1 <- tidygraph::as_tbl_graph(g) %>% ggraph::ggraph(., layout="lgl") +
    geom_edge_link(alpha=0.2, width=0.1, color="#000000") +
    geom_node_point(aes(fill=factor(enrichment_label), size=abs(logFC)), color="black", shape=21, stroke=0.1) +
    geom_node_text(aes(label=ifelse(gene %in% c("Slc40a1", "Slc48a1", "Nr1h3", "Hmox1", "Stat2", "Irf7", "Eif2ak2", "Gdf15"), gene, NA)), repel=FALSE, size=2.1, color="black") + 
    scale_fill_manual(values=c("#6d6875", "#4f6d7a", "#4a6fa5", "#161b33")) +
    scale_size_continuous(range=c(0.5, 3)) + 
    theme(aspect.ratio=1) + theme_global_set(4)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/string_intmo_d1_ko_up.pdf", width=10, height=10)

gridExtra::grid.arrange(egg::set_panel_size(g_1, width=unit(5, "cm"), height=unit(5, "cm")))

dev.off()

## intMo D1 KO DOWN 

In [ ]:
# Mapp DEA results to STRING DB
string_db_dea <- dea_res_6[["intMo"]] %>% dplyr::select(gene, avg_log2FC, p_val_adj) %>% dplyr::rename(logFC=avg_log2FC, padj=p_val_adj) %>% na.omit() %>% dplyr::filter(padj<=adj_pval_thr & logFC<=-log2FC_thr) %>% string_db$map(., "gene", removeUnmappedRows=TRUE)

In [ ]:
# Labe PPI vertices 
ppi_v <- string_db_dea %>% dplyr::distinct() %>% dplyr::rename(name=STRING_id) %>% 
    dplyr::group_by(name) %>% dplyr::mutate(n=n()) %>% dplyr::filter(n==1) %>% dplyr::select(name, gene, logFC, padj) %>% 
    dplyr::mutate(node_group=ifelse(padj<=0.05 & logFC<=-0.50, "neg", ifelse(padj<=0.05 & logFC>=0.50, "pos", "n.s.")))

In [ ]:
# Create PPI edges from DEA PPI
ppi_e <- string_db$get_interactions(string_db_dea$STRING_id)

In [ ]:
# Compute igraph object
g <- igraph::graph_from_data_frame(
    
    d=ppi_e,
    vertices=ppi_v, 
    directed=FALSE

)

In [ ]:
# Compute GO enrichment 
enrichment <- string_db$get_enrichment(V(g)$name, category="Process", methodMT="fdr", iea=TRUE)
openxlsx::write.xlsx(enrichment %>% dplyr::arrange(fdr), "result/string/scRNAseq/enrichment_intmo_d1_ko_down.xlsx")

In [ ]:
# Connected components 
min_component_size <- 4  # keep components with >= 4 vertices

g_components <- igraph::components(g)
g <- igraph::delete_vertices(g, which(g_components$csize[g_components$membership] < min_component_size))

In [ ]:
# Compute Louvain cluster on combined score 
E(g)$weight <- E(g)$combined_score

louvain <- igraph::cluster_louvain(g, weights=E(g)$weight, resolution=1.0)  # or weights=NULL
V(g)$louvain <- membership(louvain)

In [ ]:
# Compute GO enrichment for Louvain 
enrichment_louvain <- tibble::tibble(name=V(g)$name, louvain=V(g)$louvain) %>% group_by(louvain) %>%
    
    group_modify(~{
        
        enrichment <- string_db$get_enrichment(.x$name, category="Process", methodMT="fdr", iea=TRUE) %>% arrange(fdr)
        if (is.null(enrichment) | nrow(enrichment)==0) {
            
            return(tibble::tibble())
        
        } else {

            return(enrichment %>% arrange(fdr))
            
        }
        
    }
                ) %>% ungroup()

In [ ]:
openxlsx::write.xlsx(enrichment_louvain %>% dplyr::filter(fdr<=0.01), "result/string/scRNAseq/enrichment_louvain_intmo_d1_ko_down.xlsx")

In [ ]:
# Fetch GO enrichment 
enrichment_label <- enrichment_louvain %>% group_by(louvain) %>% arrange(fdr) %>% slice(1) %>% ungroup() %>% transmute(louvain, label=description)
V(g)$enrichment_label <- enrichment_label$label[match(V(g)$louvain, enrichment_label$louvain)]

In [ ]:
# Remove vertices without enrichment label 
g <- igraph::delete_vertices(g, which(is.na(V(g)$enrichment_label)))

In [ ]:
# Plot graph 
g_1 <- tidygraph::as_tbl_graph(g) %>% ggraph::ggraph(., layout="lgl") +
    geom_edge_link(alpha=0.2, width=0.1, color="#000000") +
    geom_node_point(aes(fill=factor(enrichment_label), size=abs(logFC)), color="black", shape=21, stroke=0.1) +
    geom_node_text(aes(label=ifelse(gene %in% c("Slc40a1", "Slc48a1", "Nr1h3", "Hmox1", "Stat2", "Irf7", "Eif2ak2", "Gdf15"), gene, NA)), repel=FALSE, size=2.1, color="black") + 
    scale_fill_manual(values=c("#ffcdb2", "#ffb4a2", "#e5989b", "#b5838d", "#cfe1b9", "#97a97c", "#718355", "#c0d6df", "#474973")) +
    scale_size_continuous(range=c(0.5, 3)) + 
    theme(aspect.ratio=1) + theme_global_set(4)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/string_intmo_d1_ko_down.pdf", width=10, height=10)

gridExtra::grid.arrange(egg::set_panel_size(g_1, width=unit(5, "cm"), height=unit(5, "cm")))

dev.off()

In [ ]:
e <- igraph::as_data_frame(g, what="edges")
v <- igraph::as_data_frame(g, what="vertices")

In [ ]:
v_1 <- v %>% dplyr::filter(louvain==2) %>% dplyr::select(name, gene, louvain) %>% dplyr::rename(target_name=name, target_gene=gene, target_louvain=louvain)
e_1 <- e %>% dplyr::select(from, to) %>% dplyr::filter(from %in% v_1$target_name | to %in% v_1$target_name) %>% dplyr::filter(!(from %in% v_1$target_name & to %in% v_1$target_name))

In [ ]:
v_2 <- v %>% dplyr::filter(louvain!=2) %>% dplyr::select(name, gene, louvain) %>% dplyr::rename(source_name=name, source_gene=gene, source_louvain=louvain)

In [ ]:
connect <- rbind(
    
    dplyr::left_join(v_1, e_1, by=c("target_name"="from")) %>% dplyr::mutate(from=target_name), 
    dplyr::left_join(v_1, e_1, by=c("target_name"="to"))  %>% dplyr::mutate(to=target_name)
    
) %>% na.omit() %>% dplyr::distinct() %>% dplyr::mutate(source_name=ifelse(target_name!=to, to, from)) %>% dplyr::select(-from, -to) %>% dplyr::left_join(., v_2, by = join_by(source_name)) 

## intMo

In [ ]:
# Mapp DEA results to STRING DB
string_db_dea <- dea_res_6[["intMo"]] %>% dplyr::select(gene, avg_log2FC, p_val_adj) %>% dplyr::rename(logFC=avg_log2FC, padj=p_val_adj) %>% na.omit() %>% dplyr::filter(padj<=adj_pval_thr & abs(logFC)>=log2FC_thr) %>% string_db$map(., "gene", removeUnmappedRows=TRUE)

In [ ]:
# Labe PPI vertices 
ppi_v <- string_db_dea %>% dplyr::distinct() %>% dplyr::rename(name=STRING_id) %>% 
    dplyr::group_by(name) %>% dplyr::mutate(n=n()) %>% dplyr::filter(n==1) %>% dplyr::select(name, gene, logFC, padj) %>% 
    dplyr::mutate(node_group=ifelse(padj<=0.05 & logFC<=-0.50, "IFNAR_fl_CpG_D1", ifelse(padj<=0.05 & logFC>=0.50, "IFNAR_fl_LysM_cre_CpG_D1", "n.s.")))

In [ ]:
# Create PPI edges from DEA PPI
ppi_e <- string_db$get_interactions(string_db_dea$STRING_id)

In [ ]:
# Compute igraph object
g <- igraph::graph_from_data_frame(
    
    d=ppi_e,
    vertices=ppi_v, 
    directed=FALSE

)

In [ ]:
# Connected components 
min_component_size <- 4  # keep components with >= 4 vertices

g_components <- igraph::components(g)
g <- igraph::delete_vertices(g, which(g_components$csize[g_components$membership] < min_component_size))

In [ ]:
# Compute Louvain cluster on combined score 
E(g)$weight <- E(g)$combined_score

louvain <- igraph::cluster_louvain(g, weights=E(g)$weight, resolution=1.0)  # or weights=NULL
V(g)$louvain <- membership(louvain)

In [ ]:
# Compute GO enrichment for Louvain 
enrichment_louvain <- tibble::tibble(name=V(g)$name, louvain=V(g)$louvain) %>% group_by(louvain) %>%
    
    group_modify(~{
        
        enrichment <- string_db$get_enrichment(.x$name, category="Process", methodMT="fdr", iea=TRUE) %>% arrange(fdr)
        if (is.null(enrichment) | nrow(enrichment)==0) {
            
            return(tibble::tibble())
        
        } else {

            return(enrichment %>% arrange(fdr))
            
        }
        
    }
                ) %>% ungroup()

In [ ]:
# Fetch GO enrichment 
enrichment_label <- enrichment_louvain %>% group_by(louvain) %>% arrange(fdr) %>% slice(1) %>% ungroup() %>% transmute(louvain, label=description)
V(g)$enrichment_label <- enrichment_label$label[match(V(g)$louvain, enrichment_label$louvain)]

In [ ]:
# Remove vertices without enrichment label 
g <- igraph::delete_vertices(g, which(is.na(V(g)$enrichment_label)))

In [ ]:
# Plot graph 
g_1 <- tidygraph::as_tbl_graph(g) %>% ggraph::ggraph(., layout="lgl") +
    geom_edge_link(alpha=0.2, width=0.1, color="#000000") +
    geom_node_point(aes(fill=node_group, size=abs(logFC)), color="black", shape=21, stroke=0.1) +
    geom_node_text(aes(label=ifelse(gene %in% c("Slc40a1", "Slc48a1", "Nr1h3", "Hmox1", "Stat2", "Irf7", "Eif2ak2", "Gdf15"), gene, NA)), repel=FALSE, size=2.1, color="black") + 
    scale_fill_manual(values=color$sample_group) +
    scale_size_continuous(range=c(0.5, 3)) + 
    theme(aspect.ratio=1) + theme_global_set(4)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/string_intmo_d1.pdf", width=10, height=10)

gridExtra::grid.arrange(egg::set_panel_size(g_1, width=unit(5, "cm"), height=unit(5, "cm")))

dev.off()

In [ ]:
data <- rbind(
    
    openxlsx::read.xlsx("result/string/scRNAseq/enrichment_intmo_d1_ko_down.xlsx") %>% dplyr::mutate(fill="IFNAR_fl_CpG_D1"), 
    openxlsx::read.xlsx("result/string/scRNAseq/enrichment_intmo_d1_ko_up.xlsx") %>% dplyr::mutate(fill="IFNAR_fl_LysM_cre_CpG_D1")

)

In [ ]:
go_select <- c(

    "Response to interferon-alpha", 
    "Response to interferon-beta", 

    "Positive regulation of tumor necrosis factor production", 

    "Response to stress", 
    
    "Cellular iron ion homeostasis", 
    "Iron ion transport", 

    "Response to interferon-gamma", 
    "Peptide antigen assembly with MHC class II protein complex"
    
)

In [ ]:
data %>% dplyr::filter(description %in% go_select)

In [ ]:
data <- data %>% dplyr::filter(description %in% go_select) %>% dplyr::mutate(score=-log10(fdr)) %>% dplyr::group_by(fill) %>% dplyr::arrange(score) %>% dplyr::ungroup() %>% 
    dplyr::mutate(description=factor(description, levels=rev(description)))

In [ ]:
# grn_p_1 <- ggplot(data, aes(x=score, y=description, fill=fill)) + 
#     geom_bar(stat="identity", width=0.8, color="black", linewidth=0.1) + 
#     scale_fill_manual(values=c(color$sample_group, "n.s."="#7f7f7f")) + theme_global_set(4)

In [ ]:
# pdf("result/figures/1_scRNAseq/panel_5/enrichment_intmo_d1.pdf", width=4, height=8)

# set_height <- 2.00 
# set_width <- 1

# grn_p_1 + ggh4x::force_panelsizes(cols=unit(set_width, "cm"), total_height=unit(set_height, "cm"))

# dev.off()

## Genotype comparison - M5 GO:BP

In [ ]:
compute_gsea <- FALSE
celltype_select <- c("intMo")

In [ ]:
if(compute_gsea) {

    gsea_res <- lapply(celltype_select, function(i) {
    
        gsea_res_1 <- gsea(dea_res_5[[i]], category="M5", subcategory="GO:BP", gene_set=NULL) %>% dplyr::mutate(col_split="Ctl", col_label=i)
        gsea_res_2 <- gsea(dea_res_6[[i]], category="M5", subcategory="GO:BP", gene_set=NULL) %>% dplyr::mutate(col_split="D1", col_label=i)
        gsea_res_3 <- gsea(dea_res_7[[i]], category="M5", subcategory="GO:BP", gene_set=NULL) %>% dplyr::mutate(col_split="D3", col_label=i)
    
        gsea_res <- rbind(gsea_res_1, gsea_res_2, gsea_res_3)
    
        return(gsea_res)
        
    }
          )

    saveRDS(gsea_res, "result/gsea/scRNAseq/gsea_ctl_d1_d3_wt_vs_ko_m5_gobp.rds")

} else {

    gsea_res <- readRDS("result/gsea/scRNAseq/gsea_ctl_d1_d3_wt_vs_ko_m5_gobp.rds")
    
}

In [ ]:
is.na(gsea_res$leadingEdge)

In [ ]:
gsea_res <- do.call(rbind, gsea_res)

In [ ]:
gsea_res[gsea_res$pathway %in% grep("IRON", gsea_res$pathway, value=TRUE), ] %>% dplyr::filter(pval<=0.05)

In [ ]:
grep("IRON", gsea_res %>% dplyr::pull(pathway), value=TRUE)

In [ ]:
gs_name_select <- gsea_res %>% dplyr::filter(padj<=0.05 & abs(NES)>=1) %>% dplyr::pull(pathway)
# gsea_res <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select)

In [ ]:
grep("IRON", gsea_res$pathway, value=TRUE)

In [ ]:
mat_1 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=padj,
        values_fill=1
    
    )

In [ ]:
mat_2 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=NES,
        values_fill=0
    
    )

In [ ]:
col_label <- mat_1$col_label
col_split <- mat_1$col_split

In [ ]:
mat_1 <- mat_1 %>% dplyr::select(-col_label, -col_split) %>% t()
mat_2 <- mat_2 %>% dplyr::select(-col_label, -col_split) %>% t()

In [ ]:
# mat_1 <- mat_1[gs_name_order, ]
# mat_2 <- mat_2[gs_name_order, ]

In [ ]:
col_label[col_label=="cDC.mig."] <- "cDC mig."

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/hm_gsea_ctl_d1_d3_wt_vs_ko_m5_gobp.pdf", width=8, height=100)

source("bin/gsea_pl.R")
gsea_hm(mat_1, mat_2, col_label=col_label, col_split=col_split, row_split=NULL, pathway_suffix="HALLMARK", color_neg=color$sample_group["IFNAR_fl_CpG_D1"], color_pos=color$sample_group["IFNAR_fl_LysM_cre_CpG_D1"], fontsize_select=2)

dev.off()

## Violine gene expression plots

In [ ]:
gene_select <- c("Ly6c2", "Spic", "Cd163", "Gdf15", "Ccl2", "Slfn4", "Bcl2l1", "Hmox1", "Jun")
celltype_select <- c("cMo", "intMo", "RPM")

In [ ]:
mat <- cbind(so@meta.data[, c("sample_group", "celltype_low"), drop=FALSE], t(GetAssayData(so, assay="RNA", slot="data")[gene_select, ])) %>% 
    pivot_longer(cols=-c(sample_group, celltype_low), names_to="gene", values_to="expression")

In [ ]:
map <- data.frame(
    
    sample_group=c("IFNAR_fl_Baseline_D0", "IFNAR_fl_LysM_cre_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_CpG_D3", "IFNAR_fl_LysM_cre_CpG_D3"),
    time_point=c("Ctl", "Ctl", "D1", "D1", "D3", "D3"),
    genotype=c("+/+", "cre/+", "+/+", "cre/+", "+/+", "cre/+")
)

mat <- mat %>% dplyr::left_join(., map, by=join_by(sample_group)) %>% 
    dplyr::mutate(gene=factor(gene, levels=gene_select))

In [ ]:
genotype <- c("#732E8F", "#D390BF", "#FA8775")
names(genotype) <- c("WT", "+/+", "cre/+")

In [ ]:
options(repr.plot.width=5, repr.plot.height=7.5)

exp_p <- ggplot(mat %>% dplyr::filter(celltype_low %in% celltype_select), aes(x=time_point, y=expression, color=sample_group, fill=sample_group)) + 

        geom_jitter(position=position_jitterdodge(jitter.width=0.20, dodge.width=1.0), shape=21, stroke=0.1, size=0.75, show.legend=FALSE, color="black") + 
        # geom_boxplot(position=position_dodge(width=1.0), width=0.20, color="black", alpha=1, outlier.color=NA) + 
        
        xlab("") + ylab("log(CPTT)") + 
        ggtitle("Spic") + 
        facet_grid(gene~celltype_low, scales="free") + 
        scale_color_manual(values=color$sample_group) + 
        scale_fill_manual(values=color$sample_group)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/exp_myeloid.pdf", width=3, height=8)

set_height <- 1 * length(gene_select)
set_width <- 3 * length(celltype_select)

exp_p + ggh4x::force_panelsizes(cols=unit(set_width, "mm"), total_height=unit(set_height, "cm"))

dev.off()

# Gene regulatory network analysis 

In [ ]:
auc_mtx_3 <- read.csv("result/scenic/scRNAseq/res_3/auc_mtx.csv", row.names=1) %>% t()
rownames(auc_mtx_3) <- gsub("\\.", "", rownames(auc_mtx_3))

In [ ]:
so_3 <- subset(so, subset=cell_id %in% colnames(auc_mtx_3))
auc_mtx_3 <- auc_mtx_3[, colnames(so_3)]
so_3[["AUC"]] <- CreateAssayObject(data=auc_mtx_3[, colnames(so_3)], min.cells=0, min.features=0)

## Jitter plot 

In [ ]:
mat <- cbind(so_3@meta.data, t(auc_mtx_3["Spic", , drop=FALSE]))

In [ ]:
auc_p_1 <- ggplot(mat %>% dplyr::filter(celltype_low %in% "intMo"), aes(x=genotype, y=Spic, color=sample_group, fill=sample_group)) + 
         
        geom_jitter(position=position_jitterdodge(jitter.width=0.75, dodge.width=1.0), shape=21, stroke=0.1, size=0.75, show.legend=FALSE, color="black") + 
        geom_boxplot(color="black", width=0.50, box.linewidth=0.1, whisker.linewidth=0.1, median.linewidth=0.1, outliers=FALSE) +
        xlab("") + ylab("AUC") +
        ylim(c(0, 0.30)) + 
        ggtitle("Spic") + 
        facet_grid(celltype_low~dpi) + 
        scale_color_manual(values=color$sample_group) + 
        scale_fill_manual(values=color$sample_group) + theme_global_set(4)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/auc_spic_myeloid.pdf", width=4, height=8)

set_height <- 1.5 * length(celltype_select)
set_width <- 0.75

exp_p + ggh4x::force_panelsizes(cols=unit(set_width, "cm"), total_height=unit(set_height, "cm"))

dev.off()

## Bar plot differential AUC

In [ ]:
grn_wilcox_res <- readRDS("result/scenic/scRNAseq/auc_wilcox_ery_ifnar_fl_lysm_cre_cpg_d1_vs_ifnar_fl_cpg_d1.rds")

In [ ]:
data <- grn_wilcox_res[["intMo"]]

In [ ]:
label <- c(
    
    data %>% arrange(p_val_adj) %>% dplyr::filter(sign(avg_log2FC)==-1 & rank_1 <= 50 & p_val_adj<=0.01) %>% dplyr::slice_head(n=5) %>% dplyr::pull(module), 
    data %>% arrange(p_val_adj) %>% dplyr::filter(sign(avg_log2FC)==+1 & rank_2 <= 50 & p_val_adj<=0.01) %>% dplyr::slice_head(n=5) %>% dplyr::pull(module)

)

In [ ]:
data <- data %>% dplyr::filter(module %in% label) %>% dplyr::mutate(score=-sign(avg_log2FC)*log10(p_val_adj)) %>% dplyr::arrange(score) %>% 
    dplyr::mutate(fill=ifelse(sign(score)==+1, "IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_CpG_D1")) %>% 
    dplyr::mutate(module=factor(module, levels=rev(module)))

In [ ]:
grn_p_1 <- ggplot(data, aes(x=score, y=module, fill=fill)) + 
    geom_bar(stat="identity", width=0.8, color="black", linewidth=0.1) + 
    scale_fill_manual(values=c(color$sample_group, "n.s."="#7f7f7f")) + theme_global_set(4)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/auc_diff_intmo_d1.pdf", width=4, height=8)

set_height <- 2.00 * length(celltype_select)
set_width <- 1

grn_p_1 + ggh4x::force_panelsizes(cols=unit(set_width, "cm"), total_height=unit(set_height, "cm"))

dev.off()

# CCI DEA 

In [ ]:
# Load source 
source("bin/cci_pp.R")
source("bin/cci_pl.R")

# Parameter 
pval_prob_thr <- 0.05
p_val_adj_dea <- 0.05
pathway_name_thr <- c("ICAM", "VCAM", "SIRP", "KIT", "TGFb", "TNF", "CXCL", "CCL")

# CellChat results
lr_res <- read.csv("result/cci/lr_res.csv", row.names=1)

# Add reversed Cell-cell interactions 
db <- CellChat::CellChatDB.mouse

# Impute custom interactions
options(stringsAsFactors=FALSE)
interaction <- read.csv("data/reference/cellchat/interaction.csv", row.names=1, na.strings=c(""))
interaction[is.na(interaction)] <- ""
db$interaction <- rbind(db$interaction, interaction)

# Get cell-cell contact interaction 
db <- CellChat::subsetDB(db, search=c("Cell-Cell Contact"), key="annotation")

# Subset lr results for cell-cell contact forward 
lr_res_ccc_fwd <- lr_res %>% dplyr::filter(lr_res$interaction_name %in% db$interaction$interaction_name)

# Reverse cell-cell contact 
lr_res_ccc_rev <- data.frame(

    source=lr_res_ccc_fwd$target, 
    target=lr_res_ccc_fwd$source, 
    interaction_name=paste0(lr_res_ccc_fwd$receptor_symbol, "_", lr_res_ccc_fwd$ligand_symbol), 
    prob=lr_res_ccc_fwd$prob, 
    pval=lr_res_ccc_fwd$pval, 
    pathway_name=lr_res_ccc_fwd$pathway_name, 
    ligand_symbol=lr_res_ccc_fwd$receptor_symbol, 
    receptor_symbol=lr_res_ccc_fwd$ligand_symbol, 
    sample_group=lr_res_ccc_fwd$sample_group
    
)

# Combine results 
lr_res_ccc <- rbind(lr_res_ccc_fwd, lr_res_ccc_rev)

# Assemble LR results 
lr_res <- rbind(lr_res[!lr_res$interaction_name %in% lr_res_ccc$interaction_name, ], lr_res_ccc)

# Subset by interaction pathway
lr_res <- lr_res[lr_res$pathway_name %in% pathway_name_thr, ]
lr_res$pathway_name <- factor(lr_res$pathway_name, levels=pathway_name_thr)

# DEA results 

In [ ]:
dea_mat_5 <- dea_res_5[c("cMo", "intMo", "RPM", "cDC2", "Basophil", "MEP", "Proerythroblast", "Erythroblast")]
dea_mat_5 <- lapply(dea_mat_5, function(x) {list("Ctl"=x)})

In [ ]:
dea_mat_6<- dea_res_6[c("cMo", "intMo", "RPM", "cDC2", "Basophil", "MEP", "Proerythroblast", "Erythroblast")]
dea_mat_6 <- lapply(dea_mat_6, function(x) {list("D1"=x)})

In [ ]:
dea_mat_7 <- dea_res_7[c("cMo", "intMo", "RPM", "cDC2", "Basophil", "MEP", "Proerythroblast", "Erythroblast")]
dea_mat_7 <- lapply(dea_mat_7, function(x) {list("D3"=x)})

In [ ]:
dea_mat <- Map(c, dea_mat_5, dea_mat_6, dea_mat_7)

## PTPG B (+/+) vs D1 (+/+) and D3 (+/+)

## cMo

In [ ]:
source <- c("cMo")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_cmo_mep_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("cMo")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_cmo_proeb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("cMo")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_cmo_eb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

## intMo

In [ ]:
source <- c("intMo")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_intmo_mep_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("intMo")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_intmo_proeb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("intMo")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_intmo_eb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

## RPM

In [ ]:
source <- c("RPM")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_rpm_mep_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("RPM")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_rpm_proeb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("RPM")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_rpm_eb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

# Interaction plots

In [ ]:
sources <- c("RPM", "intMo", "cMo")
targets <- c("MEP", "Proerythroblast", "Erythroblast")

In [ ]:
cco_1 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_baseline_d0.rds")
cco_2 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d1.rds")
cco_3 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d3.rds")
cco_4 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_baseline_d0.rds")
cco_5 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_cpg_d1.rds")
cco_6 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_cpg_d3.rds")

In [ ]:
cco_1 <- subsetCellChat(object=cco_1, idents.use=c(sources, targets))
cco_2 <- subsetCellChat(object=cco_2, idents.use=c(sources, targets))
cco_3 <- subsetCellChat(object=cco_3, idents.use=c(sources, targets))
cco_4 <- subsetCellChat(object=cco_4, idents.use=c(sources, targets))
cco_5 <- subsetCellChat(object=cco_5, idents.use=c(sources, targets))
cco_6 <- subsetCellChat(object=cco_6, idents.use=c(sources, targets))

In [ ]:
levels(cco_1@idents) <- c(targets, sources)
levels(cco_2@idents) <- c(targets, sources)
levels(cco_3@idents) <- c(targets, sources)
levels(cco_4@idents) <- c(targets, sources)
levels(cco_5@idents) <- c(targets, sources)
levels(cco_6@idents) <- c(targets, sources)

In [ ]:
color_use <- color$celltype_low[names(color$celltype_low) %in% c(sources, targets)]

In [ ]:
cco_list <- list(cco_1, cco_2, cco_3, cco_4, cco_5, cco_6)

q_prob_thr <- quantile(c(cco_1@net$prob, cco_2@net$prob, cco_3@net$prob), 0.90)
for (i in seq_along(cco_list)) {
    
    prob_arr <- cco_list[[i]]@net$prob
    prob_arr[prob_arr > q_prob_thr] <- q_prob_thr
    cco_list[[i]]@net$prob <- prob_arr
    
}

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cp_cci_net_ctl_ko.pdf", width=2.0, height=2.0)

    replayPlot(netVisual_chord_gene(cco_list[[4]], slot.name="net", sources.use=sources, targets.use=targets, signaling=pathway_name_thr, color.use=color_use, title.name=NULL, show.legend=FALSE, small.gap=1, big.gap=5, lab.cex=0.5))

dev.off()

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cp_cci_net_d1_ko.pdf", width=2.0, height=2.0)

    replayPlot(netVisual_chord_gene(cco_list[[5]], slot.name="net", sources.use=sources, targets.use=targets, signaling=pathway_name_thr, color.use=color_use, title.name=NULL, show.legend=FALSE, small.gap=1, big.gap=5, lab.cex=0.5))

dev.off()

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cp_cci_net_d3_ko.pdf", width=2.0, height=2.0)

    replayPlot(netVisual_chord_gene(cco_list[[6]], slot.name="net", sources.use=sources, targets.use=targets, signaling=pathway_name_thr, color.use=color_use, title.name=NULL, show.legend=FALSE, small.gap=1, big.gap=5, lab.cex=0.5))

dev.off()

# Feature plots

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/fp_selected_genes.pdf", width=2*2.5, height=5*2.5)

gridExtra::grid.arrange(
    
    fplot(so, features="Ly6c2", slot="data", restrict=c("sample_group", "IFNAR_fl_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Ly6c2", slot="data", restrict=c("sample_group", "IFNAR_fl_LysM_cre_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Spic", slot="data", restrict=c("sample_group", "IFNAR_fl_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Spic", slot="data", restrict=c("sample_group", "IFNAR_fl_LysM_cre_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Vcam1", slot="data", restrict=c("sample_group", "IFNAR_fl_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Vcam1", slot="data", restrict=c("sample_group", "IFNAR_fl_LysM_cre_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Gdf15", slot="data", restrict=c("sample_group", "IFNAR_fl_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Gdf15", slot="data", restrict=c("sample_group", "IFNAR_fl_LysM_cre_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Ccl2", slot="data", restrict=c("sample_group", "IFNAR_fl_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Ccl2", slot="data", restrict=c("sample_group", "IFNAR_fl_LysM_cre_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), ncol=2
    
)

dev.off()